# 030-rede2018_codifica_receitas

In [1]:
ano_eleicao = '2018'

In [2]:
dbschema = f'rede{ano_eleicao}'
dbschema_tse = f'tse{ano_eleicao}'
table_receitas = f'{dbschema}.receitas_{ano_eleicao}'
table_receitas_candidatos = f'{dbschema_tse}.receitas_candidatos_{ano_eleicao}'
table_receitas_candidatos_doador_originario = f'{dbschema_tse}.receitas_candidatos_doador_originario_{ano_eleicao}'
table_receitas_orgaos_partidarios = f'{dbschema_tse}.receitas_orgaos_partidarios_{ano_eleicao}'
table_receitas_orgaos_partidarios_doador_originario= f'{dbschema_tse}.receitas_orgaos_partidarios_doador_originario_{ano_eleicao}'
table_consulta_cand = f'{dbschema_tse}.consulta_cand_{ano_eleicao}'
table_candidaturas = f"{dbschema}.candidaturas_{ano_eleicao}"
table_municipios = f"tse{ano_eleicao}.municipios_{ano_eleicao}"
table_partidos = f'{dbschema_tse}.partidos_{ano_eleicao}'
table_orgaos_partidarios =  f"{dbschema}.orgaos_partidarios_{ano_eleicao}"
table_origem_receitas = f'tse.origem_receitas'
table_fonte_receitas = f'tse.fonte_receitas'

In [3]:
import os
import sys
sys.path.append('../')
import mod_tse as mtse
home = os.environ["HOME"]

In [4]:
mtse.execute_query(f'CREATE SCHEMA IF NOT EXISTS {dbschema};')

## CODIFICA DESCRIÇÃO, CÓDIGO E SIGLA ORIGEM DAS RECEITAS

In [5]:
query_update_origem_receita = f"""
update {table_receitas} as r
    set receita_origem_sg = o.sg_origem_receita,
        receita_origem_cd = o.cd_origem_receita,
        receita_origem_ds = o.ds_origem_receita
from {table_origem_receitas} as o
where upper(r.receita_origem_cd) = upper(o.cd_origem_receita)
or upper(r.receita_origem_ds) = upper(o.tx_origem_receita)
;
"""

mtse.execute_query(query_update_origem_receita)

## CODIFICA DESCRIÇÃO E CÓDIGO DA FONTE DA RECEITA

In [6]:
query_update_fonte_receita = f"""
update {table_receitas} as r
    set receita_fonte_cd = f.cd_fonte_receita,
        receita_fonte_ds = f.ds_fonte_receita
from {table_fonte_receitas} as f
where upper(r.receita_fonte_cd) = upper(f.cd_fonte_receita)
or upper(r.receita_fonte_ds) = upper(f.tx_fonte_receita)
;
"""

mtse.execute_query(query_update_fonte_receita)

## CODIFICA UF E MUNICÍPIO DO DOADOR 

In [7]:
siglas_uf = list(mtse.unidades_da_federacao.keys())
siglas_uf="'" +"','".join(siglas_uf) +"'"

query_update_uf_doador_OP = f"""
update {table_receitas} as r
    set doador_uf = m.sg_uf,
        doador_ue_nome = m.nm_municipio
from {table_municipios} as m
where 
m.cd_municipio = doador_ue
;
"""

mtse.execute_query(query_update_uf_doador_OP)

## CODIFICA SIGLA/NUMERO DO PARTIDO DO DOADOR 

In [8]:
query_update_doador_partido_sg = f"""
with partidos as
(
select sg_partido, nr_partido, nm_partido
from {table_partidos}
group by sg_partido, nr_partido, nm_partido
)
update {table_receitas} as r
    set 
        doador_partido_sg   = p.sg_partido,
        doador_partido_nr   = p.nr_partido,
        doador_partido_nome = p.nm_partido        
from partidos as p
where 
upper(r.doador_partido_sg) = upper(p.sg_partido)
or
upper(r.doador_partido_nr) = upper(p.nr_partido)
;
"""

mtse.execute_query(query_update_doador_partido_sg)

## CODIFICA SIGLA/NUMERO DO PARTIDO DO RECEPTOR

In [9]:
query_update_receptor_partido = f"""
with partidos as
(
select sg_partido, nr_partido, nm_partido
from {table_partidos}
group by sg_partido, nr_partido, nm_partido
)
update {table_receitas} as r
    set 
        receptor_partido_sg   = p.sg_partido,
        receptor_partido_nr   = p.nr_partido,
        receptor_partido_nome = p.nm_partido        
from partidos as p
where 
upper(r.receptor_partido_sg) = upper(p.sg_partido)
or
upper(r.receptor_partido_nr) = upper(p.nr_partido)
;
"""

mtse.execute_query(query_update_receptor_partido)